In [1]:
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

In [2]:
DATA_URL = 'https://github.com/gcaff/COVID19-RD/raw/master/data/covid_data_rd.csv'
DATA_TEMPLATE_URL = 'https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/master/latam_covid_19_data/templates/daily_report.csv'
PATH_CSV = "utils/scripts/data_collection/data/republica_dominicana_temporal/"
PATH_DSRP_DAILY_REPORTS = 'latam_covid_19_data/daily_reports/'
LAST_UPDATE = datetime.today().isoformat()
DICT_PLACES={'Azua':'DO-02',
            'Baoruco':'DO-03',
            'Barahona':'DO-04',
            'Dajabón':'DO-05',
            'Distrito Nacional':'DO-06',
            'Duarte':'DO-08',
            'El Seibo':'DO-09',
            'Elías Piña':'DO-30',
            'Espaillat':'DO-10',
            'Hato Mayor':'DO-11',
            'Hermanas Mirabal':'DO-07',
            'Independencia':'DO-12',
            'La Altagracia':'DO-13',
            'La Romana':'DO-14',
            'La Vega':'DO-28',
            'María Trinidad Sánchez':'DO-15',
            'Monseñor Nouel':'DO-29',
            'Monte Cristi':'DO-16',
            'Monte Plata':'DO-17', 
            'Pedernales':'DO-18', 
            'Peravia':'DO-19',
            'Puerto Plata':'DO-20',
            'Samaná':'DO-21',
            'San Cristóbal':'DO-31', 
            'San José de Ocoa':'DO-22',
            'San Juan':'DO-23', 
            'San Pedro de Macorís':'DO-24',
            'Sánchez Ramírez':'DO-25', 
            'Santiago':'DO-26', 
            'Santiago Rodríguez':'DO-32', 
            'Santo Domingo':'DO-01',
            'Valverde':'DO-27',
            'No especificado':np.nan
            }

In [36]:
df=pd.read_csv(DATA_URL, usecols=['fecha','provincia','casos_acum','defun_acum','recuperados'],encoding='latin-1',parse_dates=['fecha'],dayfirst=True)
df=df.fillna(0)
df['ISO 3166-2 Code']=df['provincia'].map(DICT_PLACES)
df=df.dropna()
df

,fecha,provincia,casos_acum,recuperados,defun_acum,ISO 3166-2 Code
0,2020-03-17,Azua,0,0.0,0,DO-02
1,2020-03-17,Baoruco,0,0.0,0,DO-03
2,2020-03-17,Barahona,0,0.0,0,DO-04
3,2020-03-17,Dajabón,0,0.0,0,DO-05
4,2020-03-17,Distrito Nacional,14,0.0,0,DO-06
...,...,...,...,...,...,...
13954,2021-05-15,Monseñor Nouel,3541,3079.0,51,DO-29
13955,2021-05-15,Monte Plata,1393,898.0,32,DO-17
13956,2021-05-15,Hato Mayor,902,600.0,17,DO-11
13957,2021-05-15,San José de Ocoa,847,662.0,15,DO-22


In [37]:
df_template=pd.read_csv(DATA_TEMPLATE_URL)
df_template=df_template.fillna(0)
#df_template['Confirmed']=df_template['Confirmed'].astype(int)
#df_template['Deaths']=df_template['Deaths'].astype(int)
#df_template['Recovered']=df_template['Recovered'].astype(int)
df_template['Last Update']=LAST_UPDATE
df_template.head()

,ISO 3166-2 Code,Country,Subdivision,Last Update,Confirmed,Deaths,Recovered
0,AR-B,Argentina,Buenos Aires,2021-05-21T00:44:59.404098,0.0,0.0,0.0
1,AR-K,Argentina,Catamarca,2021-05-21T00:44:59.404098,0.0,0.0,0.0
2,AR-H,Argentina,Chaco,2021-05-21T00:44:59.404098,0.0,0.0,0.0
3,AR-U,Argentina,Chubut,2021-05-21T00:44:59.404098,0.0,0.0,0.0
4,AR-C,Argentina,Ciudad Autonoma de Buenos Aires,2021-05-21T00:44:59.404098,0.0,0.0,0.0


In [40]:
df[((df['ISO 3166-2 Code']=='DO-02')&(df['fecha']=='2021-05-10'))]['casos_acum'].values[0]

3408

In [10]:
lista=df['ISO 3166-2 Code'].unique()

In [44]:
day='2021-05-10'
for country_code in lista:
    value=df[((df['ISO 3166-2 Code']==country_code)&(df['fecha']==day))]['casos_acum'].values[0]
    condition=(df['ISO 3166-2 Code']==country_code)
    df_template.loc[condition,'Confirmed']=int(value)    
    
    value=df[((df['ISO 3166-2 Code']==country_code)&(df['fecha']==day))]['defun_acum'].values[0]
    condition=(df['ISO 3166-2 Code']==country_code)
    df_template.loc[condition,'Deaths']=int(value)
    
    value=df[((df['ISO 3166-2 Code']==country_code)&(df['fecha']==day))]['recuperados'].values[0]
    condition=(df['ISO 3166-2 Code']==country_code)
    df_template.loc[condition,'Recovered']=int(value)
    

In [47]:
df_filtered=df_template.loc[df_template['ISO 3166-2 Code'].str.contains('DO-')]
df_filtered['Confirmed']=df_filtered['Confirmed'].astype(int)
df_filtered['Deaths']=df_filtered['Deaths'].astype(int)
df_filtered['Recovered']=df_filtered['Recovered'].astype(int)
df_filtered

<ipython-input-47-fd49e9ec1ada>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Confirmed']=df_filtered['Confirmed'].astype(int)
<ipython-input-47-fd49e9ec1ada>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Deaths']=df_filtered['Deaths'].astype(int)
<ipython-input-47-fd49e9ec1ada>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,ISO 3166-2 Code,Country,Subdivision,Last Update,Confirmed,Deaths,Recovered
132,DO-02,Dominican Republic,Azua,2021-05-21T00:44:59.404098,80276,549,74925
133,DO-03,Dominican Republic,Baoruco,2021-05-21T00:44:59.404098,6572,217,6044
134,DO-04,Dominican Republic,Barahona,2021-05-21T00:44:59.404098,899,11,582
135,DO-05,Dominican Republic,Dajabon,2021-05-21T00:44:59.404098,415,6,329
136,DO-06,Dominican Republic,Distrito Nacional,2021-05-21T00:44:59.404098,5375,135,3408
137,DO-08,Dominican Republic,Duarte,2021-05-21T00:44:59.404098,886,17,595
138,DO-09,Dominican Republic,El Seibo,2021-05-21T00:44:59.404098,2448,46,1835
139,DO-30,Dominican Republic,Elias Pina,2021-05-21T00:44:59.404098,1070,14,834
140,DO-10,Dominican Republic,Espaillat,2021-05-21T00:44:59.404098,9687,73,8395
141,DO-11,Dominican Republic,Hato Mayor,2021-05-21T00:44:59.404098,8141,142,7122
